# 출력층 설계 (Output layer)

In [1]:
# !pip3 install torch torchvision torchaudio

### 소프트맥스 오버플로우 방지

In [2]:
import numpy as np

def softmax(x):
    exp_z = np.exp(x)
    return exp_z / np.sum(exp_z)

def stable_softmax(z):
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z)

x = np.array([1000, 1001, 1002])
print(softmax(x))
print(stable_softmax(x))

[nan nan nan]
[0.09003057 0.24472847 0.66524096]


C:\Users\lemon\AppData\Local\Temp\ipykernel_6532\1283896694.py:4: RuntimeWarning: overflow encountered in exp
  exp_z = np.exp(x)
C:\Users\lemon\AppData\Local\Temp\ipykernel_6532\1283896694.py:5: RuntimeWarning: invalid value encountered in divide
  return exp_z / np.sum(exp_z)


- pytorch 라이브러리 함수 사용

In [3]:
import torch
import torch.nn.functional as F     # nn: neural network package

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)   #  float로 datatype 맞춰줘야 함

softmax_output = F.softmax(x)
print(softmax_output)

sigmoid_output = torch.sigmoid(x)
print(sigmoid_output)

tensor([0.0900, 0.2447, 0.6652])
tensor([1., 1., 1.])


C:\Users\lemon\AppData\Local\Temp\ipykernel_6532\2262784125.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_output = F.softmax(x)


### 손실 함수와 연계

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# 간단한 다중 클래스 분류 모델 정의
class SimpleMultiClassModel(nn.Module):
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5, 3)

    # 순전파
    def forward(self, x):
        return self.fc(x)   # fully connected 층

# --> 구조 정의 하고 예측

# 모델, 손실함수, 최적화함수 설정
model = SimpleMultiClassModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)     

# 데이터 생성
inputs = torch.randn(4, 5)      # sample label data 4개
labels = torch.tensor([0, 2, 1, 0])

# 학습
for _ in range(10):
    preds = model(inputs)               # 순전파        model.forward(inputs)랑 똑같음
    loss = criterion(preds, labels)     # 손실 계산
    print(loss.item())                  # 손실값 출력

    optimizer.zero_grad()               # 기울기 초기화 (이전 단계에서 계산된 기울기를 0으로 초기화)
    loss.backward()                     # 손실에 대한 역전파 == 가중치 기울기 계산 (손실에 대한 역전파 수행 - 파라미터에 대한 기울기 계산)
    optimizer.step()                    # 가중치 업데이트 (계산된 기울기를 사용하여 옵티마이저가 모델 파라미터 갱신)

# 가중치가 업데이트 되면서 손실값 조금씩 줄어듦

1.0821858644485474
1.0582523345947266
1.0349880456924438
1.0123851299285889
0.990421712398529
0.9690577983856201
0.9482390284538269
0.9279109835624695
0.9080328941345215
0.8885785341262817
